# The Cholesky decomposition

In [1]:
import numba
import numpy as np
from numba import njit
from scipy.linalg import cholesky
np.set_printoptions(suppress=True, precision=4, linewidth=150)

In [2]:
xvals = np.linspace(-3, 3, 100)

Let ${\bf \Sigma}$ be an $N\,P\times N\,P$ positive definite matrix such that
$$
{\bf \Sigma} =
\begin{bmatrix}
    {\bf \sigma}_{1,1} & {\bf \sigma}_{1,2} & \cdots & {\bf \sigma}_{1,N}\\
    {\bf \sigma}_{2,1} & {\bf \sigma}_{2,2} & \cdots & {\bf \sigma}_{2,N}\\
    \vdots & \vdots & \ddots & \vdots \\
    {\bf \sigma}_{N,1} & {\bf \sigma}_{N,2} & \ldots & {\bf \sigma}_{N,N}
\end{bmatrix}
$$

We seek to decompose ${\bf \Sigma} = {\bf L\,R\,L}^\intercal$
with ${\bf R} = {\rm diag}(R_1, \ldots, R_N)$ and
${\bf L} = \{L_{i,j} : i \in \{1, \ldots N\}, j \leq i\}$ a block-lower triangular matrix with $L_{i,i} = {\bf I}$

We seek ${\bf L}$ and ${\bf R}$ such that
$$
    ({\bf L\,R\,L}^\intercal)_{i,j} = \sigma_{i,j}
$$

The choleskly decomposition yields the following algorithm

$$
\begin{aligned}
    {\bf R}_i &= \sigma_{i,i} - \sum_{k=1}^{i-1}{\bf L}_{i,k}\,{\bf R}_{k}\,{\bf L}_{i,k}^\intercal
    &\text{for }i=1,\ldots,T\\
    {\bf L}_{i,j} &= \left[\sigma_{i,j} - \sum_{k=1}^{j-1}{\bf L}_{i,k}\,{\bf R}_k\,{\bf L}_{j,k}^\intercal\right]
    &\text{for } i=2,\ldots,T\text{ and }\, j<i
\end{aligned}
$$

In [48]:
np.random.seed(314)
d = 10
S = np.random.randn(d, d)
S = S @ S.T

In [49]:
LR = np.linalg.cholesky(S)

In [50]:
S

array([[ 5.0561, -2.1211, -4.9336,  1.7228, -1.1385, -4.5112,  4.1296,  1.4875, -1.6306,  1.1243],
       [-2.1211, 10.6464,  4.9359, -4.3093, -0.2017, -1.0459, -0.481 ,  3.6758, -3.9953,  1.0951],
       [-4.9336,  4.9359, 15.9405, -1.533 ,  3.0908,  3.4337, -2.9025, -0.8334, -1.8779, -1.783 ],
       [ 1.7228, -4.3093, -1.533 ,  7.1912,  0.5188, -0.0115, -0.6576, -3.4788, -1.2517,  2.3797],
       [-1.1385, -0.2017,  3.0908,  0.5188,  7.163 ,  1.5606, -2.7542,  0.1679,  1.2635,  1.2001],
       [-4.5112, -1.0459,  3.4337, -0.0115,  1.5606,  5.6308, -4.7271, -2.4139,  3.0437, -1.037 ],
       [ 4.1296, -0.481 , -2.9025, -0.6576, -2.7542, -4.7271,  7.7946,  5.5878, -3.4981, -0.1397],
       [ 1.4875,  3.6758, -0.8334, -3.4788,  0.1679, -2.4139,  5.5878, 11.2912, -1.5058, -0.1032],
       [-1.6306, -3.9953, -1.8779, -1.2517,  1.2635,  3.0437, -3.4981, -1.5058, 11.7888,  0.6448],
       [ 1.1243,  1.0951, -1.783 ,  2.3797,  1.2001, -1.037 , -0.1397, -0.1032,  0.6448,  5.8579]])

In [51]:
LR

array([[ 2.2486,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.9433,  3.1236,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-2.1941,  0.9176,  3.2069,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.7662, -1.1482,  0.3747,  2.2683,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.5063, -0.2175,  0.6796,  0.1774,  2.5231,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-2.0063, -0.9407, -0.0327,  0.2018,  0.1295,  0.8138,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 1.8365,  0.4006,  0.2368, -0.7466, -0.6998, -0.5121,  1.7017,  0.    ,  0.    ,  0.    ],
       [ 0.6615,  1.3766, -0.2012, -1.027 ,  0.4444,  0.4319,  2.1357,  1.7082,  0.    ,  0.    ],
       [-0.7252, -1.4981, -0.6531, -0.9573,  0.4693,  0.357 , -0.949 ,  0.9282,  2.3592,  0.    ],
       [ 0.5   ,  0.5016, -0.3574,  1.1932,  0.6316,  0.1274,  0.1315, -0.3439,  1.174 ,  1.3695]])

In [52]:
R = np.diag(LR)
L = LR / np.diag(LR)

In [53]:
L

array([[ 1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.4195,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.9758,  0.2938,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.3407, -0.3676,  0.1168,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.2252, -0.0696,  0.2119,  0.0782,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.8922, -0.3012, -0.0102,  0.089 ,  0.0513,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.8168,  0.1283,  0.0738, -0.3291, -0.2774, -0.6293,  1.    ,  0.    ,  0.    ,  0.    ],
       [ 0.2942,  0.4407, -0.0627, -0.4528,  0.1761,  0.5308,  1.255 ,  1.    ,  0.    ,  0.    ],
       [-0.3225, -0.4796, -0.2036, -0.422 ,  0.186 ,  0.4387, -0.5576,  0.5433,  1.    ,  0.    ],
       [ 0.2224,  0.1606, -0.1115,  0.526 ,  0.2503,  0.1566,  0.0773, -0.2013,  0.4976,  1.    ]])

In [54]:
R

array([2.2486, 3.1236, 3.2069, 2.2683, 2.5231, 0.8138, 1.7017, 1.7082, 2.3592, 1.3695])

## Step-by-step construction

In [55]:
R_vals = np.zeros(d)
L_vals = np.zeros((d,d))

## Step 1 — construction of the first column

In [56]:
R_vals[0] = S[0,0]
R_vals[0]

5.056067919855992

In [57]:
R_vals

array([5.0561, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ])

In [58]:
L_vals[:, 0] = S[:, 0] / R_vals[0]
L_vals[:, 0]

array([ 1.    , -0.4195, -0.9758,  0.3407, -0.2252, -0.8922,  0.8168,  0.2942, -0.3225,  0.2224])

In [59]:
L_vals

array([[ 1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.4195,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.9758,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.3407,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.2252,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.8922,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.8168,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.2942,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.3225,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.2224,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]])

## Step 2 — construction of second column

In [60]:
L_vals[1,0]

-0.4195173250631725

In [61]:
R_vals[1] = S[1,1] - L_vals[1,0] * R_vals[0] * L_vals[1,0]
R_vals[1]

9.756605325989712

In [62]:
L_vals[1:, 1] = (S[1:,1] - L_vals[1:,0] * R_vals[0] * L_vals[1,0]) / R_vals[1]

In [63]:
L_vals

array([[ 1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.4195,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.9758,  0.2938,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.3407, -0.3676,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.2252, -0.0696,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.8922, -0.3012,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.8168,  0.1283,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.2942,  0.4407,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.3225, -0.4796,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.2224,  0.1606,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]])

## Further steps

In [64]:
for k in range(2, d):
    R_vals[k] = S[k, k] - (L_vals[k, :k] * R[:k] * L_vals[k, :k]).sum()
    for j in range(k, d):
        L_vals[j, k] = S[j,k] - (L_vals[k, :j] * R[:j] * L_vals[j, :j]).sum()
        L_vals[j, k] = L_vals[j,k] / R_vals[j]
    L_vals[k, k] = 1

/var/folders/vf/3b13gs616172kztlr53t6_s00000gn/T/ipykernel_23726/1761289545.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  L_vals[j, k] = L_vals[j,k] / R_vals[j]
/var/folders/vf/3b13gs616172kztlr53t6_s00000gn/T/ipykernel_23726/1761289545.py:5: RuntimeWarning: invalid value encountered in scalar divide
  L_vals[j, k] = L_vals[j,k] / R_vals[j]


In [65]:
R_vals

array([ 5.0561,  9.7566, 13.53  ,    -inf,     nan,     nan,     nan,     nan,     nan,     nan])

In [66]:
L_vals

array([[ 1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.4195,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.9758,  0.2938,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.3407, -0.3676,    -inf,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.2252, -0.0696,     inf,     nan,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.8922, -0.3012,     inf,     nan,     nan,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.8168,  0.1283,    -inf,     nan,     nan,     nan,  1.    ,  0.    ,  0.    ,  0.    ],
       [ 0.2942,  0.4407,    -inf,     nan,     nan,     nan,     nan,  1.    ,  0.    ,  0.    ],
       [-0.3225, -0.4796,    -inf,     nan,     nan,     nan,     nan,     nan,  1.    ,  0.    ],
       [ 0.2224,  0.1606,    -inf,     nan,     nan,     nan,     nan,     nan,     nan,  1.    ]])

In [47]:
L

array([[ 1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.4195,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.9758,  0.2938,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.3407, -0.3676,  0.1168,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.2252, -0.0696,  0.2119,  0.0782,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [-0.8922, -0.3012, -0.0102,  0.089 ,  0.0513,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [ 0.8168,  0.1283,  0.0738, -0.3291, -0.2774, -0.6293,  1.    ,  0.    ,  0.    ,  0.    ],
       [ 0.2942,  0.4407, -0.0627, -0.4528,  0.1761,  0.5308,  1.255 ,  1.    ,  0.    ,  0.    ],
       [-0.3225, -0.4796, -0.2036, -0.422 ,  0.186 ,  0.4387, -0.5576,  0.5433,  1.    ,  0.    ],
       [ 0.2224,  0.1606, -0.1115,  0.526 ,  0.2503,  0.1566,  0.0773, -0.2013,  0.4976,  1.    ]])